# AI-Enhanced Early Warning System for Multi-Hazard Scenarios: India-wide with Major Cities

In [ ]:

!pip install requests pandas numpy plotly folium scikit-learn matplotlib

In [ ]:

import requests
import pandas as pd
import numpy as np
import plotly.express as px
import folium
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Major Indian Cities
cities_india = {
    "Delhi": (28.6139, 77.2090),
    "Lucknow": (26.8467, 80.9462),
    "Chandigarh": (30.7333, 76.7794),
    "Shimla": (31.1048, 77.1734),
    "Srinagar": (34.0837, 74.7973),
    "Leh": (34.1526, 77.5771),
    "Jaipur": (26.9124, 75.7873),
    "Dehradun": (30.3165, 78.0322),
    "Mumbai": (19.0760, 72.8777),
    "Pune": (18.5204, 73.8567),
    "Gandhinagar": (23.2156, 72.6369),
    "Panaji": (15.4909, 73.8278),
    "Chennai": (13.0827, 80.2707),
    "Bengaluru": (12.9716, 77.5946),
    "Hyderabad": (17.3850, 78.4867),
    "Thiruvananthapuram": (8.5241, 76.9366),
    "Kolkata": (22.5726, 88.3639),
    "Patna": (25.5941, 85.1376),
    "Ranchi": (23.3441, 85.3096),
    "Bhubaneswar": (20.2961, 85.8245),
    "Guwahati": (26.1445, 91.7362),
    "Imphal": (24.8170, 93.9368),
    "Shillong": (25.5788, 91.8933),
    "Aizawl": (23.7271, 92.7176),
    "Kohima": (25.6751, 94.1086),
    "Agartala": (23.8315, 91.2868),
    "Itanagar": (27.0844, 93.6053),
    "Gangtok": (27.3389, 88.6065),
    "Bhopal": (23.2599, 77.4126),
    "Raipur": (21.2514, 81.6296),
}

# Step 2: Fetch Weather Data
data_list = []

for city, (lat, lon) in cities_india.items():
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&hourly=temperature_2m,precipitation,wind_speed_10m"
    response = requests.get(url).json()

    try:
        temp = np.mean(response['hourly']['temperature_2m'])
        rain = np.mean(response['hourly']['precipitation'])
        wind = np.mean(response['hourly']['wind_speed_10m'])
    except:
        temp, rain, wind = None, None, None

    data_list.append({
        "city": city,
        "latitude": lat,
        "longitude": lon,
        "temperature": temp,
        "rainfall": rain,
        "wind_speed": wind
    })

df = pd.DataFrame(data_list)

# Step 3: Save and Reload Dataset
path = "/content/disaster_dataset.csv"
df.to_csv(path, index=False)
print(f"Dataset saved at {path}")

Dataset saved at /content/disaster_dataset.csv


In [ ]:
# Reload
data = pd.read_csv(path)

print("\nDataset Info:")
print(data.info())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city         30 non-null     object 
 1   latitude     30 non-null     float64
 2   longitude    30 non-null     float64
 3   temperature  30 non-null     float64
 4   rainfall     30 non-null     float64
 5   wind_speed   30 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.5+ KB
None


In [ ]:
print("\nDataset Description:")
print(data.describe())


Dataset Description:
        latitude  longitude  temperature   rainfall  wind_speed
count  30.000000  30.000000    30.000000  30.000000   30.000000
mean   23.718190  82.211950    24.873730   0.411925    6.893175
std     6.118056   7.243863     4.081061   0.357970    3.922640
min     8.524100  72.636900    12.581548   0.053571    3.092857
25%    20.534925  76.995800    23.325446   0.137351    4.658929
50%    24.324250  79.378700    26.003869   0.368750    6.261905
75%    27.041400  88.545850    27.664583   0.529018    7.673958
max    34.152600  94.108600    30.513095   1.644643   23.952976


In [ ]:

print("\nMissing Values:")
print(data.isnull().sum())


Missing Values:
city           0
latitude       0
longitude      0
temperature    0
rainfall       0
wind_speed     0
dtype: int64


In [ ]:
# Fill missing values with mean for numeric columns only
numeric_cols = data.select_dtypes(include=np.number).columns
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

In [ ]:
# Step 5: Synthetic Hazard Labels
np.random.seed(42)
data["flood"] = np.random.randint(0, 2, size=len(data))
data["drought"] = np.random.randint(0, 2, size=len(data))
data["cyclone"] = np.random.randint(0, 2, size=len(data))

In [ ]:
# Step 6: Train/Test Split
X = data[["temperature", "rainfall", "wind_speed"]]
y = data[["flood", "drought", "cyclone"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ensemble Models
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

ensemble = VotingClassifier(estimators=[
    ('rf', rf), ('gb', gb)
], voting='soft')

multi_target_model = MultiOutputClassifier(ensemble)
multi_target_model.fit(X_train, y_train)

y_pred = multi_target_model.predict(X_test)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Flood", "Drought", "Cyclone"]))


Classification Report:
              precision    recall  f1-score   support

       Flood       1.00      0.50      0.67         4
     Drought       0.33      0.50      0.40         2
     Cyclone       1.00      0.40      0.57         5

   micro avg       0.71      0.45      0.56        11
   macro avg       0.78      0.47      0.55        11
weighted avg       0.88      0.45      0.57        11
 samples avg       0.50      0.50      0.50        11



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Step 7: Hazard Probabilities per City
probs = []
for _, row in data.iterrows():
    x = row[['temperature', 'rainfall', 'wind_speed']].values.reshape(1,-1)
    prob = multi_target_model.predict_proba(x)
    # Assuming the order of classes is consistent: Flood, Drought, Cyclone
    prob = [p[0][1] for p in prob]  # extract hazard probs
    probs.append(prob)

data['Flood_prob'], data['Drought_prob'], data['Cyclone_prob'] = np.array(probs).T

city_probs = data.groupby('city')[['latitude', 'longitude', 'Flood_prob','Drought_prob','Cyclone_prob']].mean().reset_index()
print("\nAverage Hazard Probabilities by City:")
display(city_probs)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/u


Average Hazard Probabilities by City:


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn

,city,latitude,longitude,Flood_prob,Drought_prob,Cyclone_prob
0,Agartala,23.8315,91.2868,0.962931,0.914579,0.030221
1,Aizawl,23.7271,92.7176,0.223550,0.105284,0.288578
2,Bengaluru,12.9716,77.5946,0.030727,0.984921,0.929855
3,Bhopal,23.2599,77.4126,0.925497,0.090589,0.832219
4,Bhubaneswar,20.2961,85.8245,0.152612,0.899407,0.908646
5,Chandigarh,30.7333,76.7794,0.062029,0.859450,0.073611
6,Chennai,13.0827,80.2707,0.085449,0.954586,0.091795
7,Dehradun,30.3165,78.0322,0.101918,0.080530,0.130330
8,Delhi,28.6139,77.2090,0.065871,0.135444,0.129357
9,Gandhinagar,23.2156,72.6369,0.025561,0.090440,0.898197


In [ ]:
# Step 8: Interactive Plotly Dashboard
fig = px.bar(
    city_probs.melt(id_vars='city', value_vars=['Flood_prob', 'Drought_prob', 'Cyclone_prob'], var_name='Hazard', value_name='Probability'),
    x='city', y='Probability', color='Hazard', barmode='group',
    title="Interactive Hazard Probabilities Across India"
)
fig.show()

In [ ]:
import folium
from folium import plugins

# Base map
india_map = folium.Map(location=[22.9734, 78.6569], zoom_start=5)

# Feature groups for disasters
disaster_layers = {
    "Flood": folium.FeatureGroup(name="Flood Risk"),
    "Drought": folium.FeatureGroup(name="Drought Risk"),
    "Cyclone": folium.FeatureGroup(name="Cyclone Risk"),
    "All": folium.FeatureGroup(name="All Risks")
}

# Add markers to each layer
for _, row in city_probs.iterrows():
    # Individual risks
    risks = {
        "Flood": row["Flood_prob"],
        "Drought": row["Drought_prob"],
        "Cyclone": row["Cyclone_prob"]
    }

    for disaster, risk_value in risks.items():
        color = "green" if risk_value <= 0.3 else "orange" if risk_value <= 0.6 else "red"
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=8,
            popup=f"{row['city']} - {disaster} Risk: {risk_value:.2f}",
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7
        ).add_to(disaster_layers[disaster])

    # Combined "All Risks" marker
    total_risk = sum(risks.values()) / 3
    all_color = "green" if total_risk <= 0.3 else "orange" if total_risk <= 0.6 else "red"
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=10,
        popup=f"{row['city']} - Total Risk: {total_risk:.2f}",
        color=all_color,
        fill=True,
        fill_color=all_color,
        fill_opacity=0.7
    ).add_to(disaster_layers["All"])

    # Extreme risk blinking alerts
    if total_risk > 0.8:
        folium.Marker(
            location=[row["latitude"], row["longitude"]],
            icon=folium.DivIcon(html=f"""
            <div style="color:red; font-weight:bold; animation: blinker 1s linear infinite;">
            ⚠️ {row['city']} Extreme Risk ({total_risk:.2f})
            </div>
            """)
        ).add_to(india_map)

# Add layers to map
for layer in disaster_layers.values():
    layer.add_to(india_map)

# Heatmap layer for overall risk
heat_data = [[row["latitude"], row["longitude"],
              (row["Flood_prob"] + row["Drought_prob"] + row["Cyclone_prob"])/3]
             for _, row in city_probs.iterrows()]
plugins.HeatMap(heat_data, radius=25, blur=15, max_zoom=10).add_to(india_map)

# Layer control
folium.LayerControl(collapsed=False).add_to(india_map)

# Legend & CSS for blinking
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 160px;
            border:2px solid grey; z-index:9999; font-size:14px; background-color:white; padding: 10px;">
<b> Risk Level </b><br>
<i style="background:green; width:15px; height:15px; float:left; margin-right:8px;"></i> Low (≤ 0.3)<br>
<i style="background:orange; width:15px; height:15px; float:left; margin-right:8px;"></i> Medium (0.3–0.6)<br>
<i style="background:red; width:15px; height:15px; float:left; margin-right:8px;"></i> High (> 0.6)<br>
</div>
<style>
@keyframes blinker { 50% { opacity: 0; } }
</style>
'''
india_map.get_root().html.add_child(folium.Element(legend_html))

india_map